In [235]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.cluster import KMeans

In [236]:
hankel = Hankel()
rank = Rank()
cluster = Cluster()
meepc = Meepc.MEEPC()

In [237]:
df = pd.read_csv('~/data/training_data.csv')
df_normal=df[df['Attack LABLE (1:No Attack, -1:Attack)']==1]
df_attack=df[df['Attack LABLE (1:No Attack, -1:Attack)']==-1]
sensors = [3,9,25,43]
df_normal = df_normal.iloc[:, sensors]
df_attack=df_attack.iloc[:,sensors]

In [276]:
sens = 2
optimal_k = 9

In [277]:
X = df_normal.iloc[:,sens].values
X = hankel.fit(X,30,0.5)
X=X.T # size--> 1069X1800
# r = rank.fit(X)
# U,Sigma,VT = np.linalg.svd(X)
# V = VT.T
# print(X.shape)
# print(U.shape)
# print(V.shape)
# X = np.matmul(X,V[:,:r])
# weight,center = meepc.fit(X)
# var1 = np.square(X-center)
# var2 = np.matmul(weight,var1.T)
# radiis_normal = np.sqrt(var2)
# kmeans,optimal_k = cluster.fit(X)
kmeans = KMeans(n_clusters=optimal_k, init='k-means++')
kmeans.fit(X)
print(optimal_k)
radiis_normal = []
weights=[]
centers=[]
clusters_R=[]
clusters_V=[]
for i in range(optimal_k):
    cluster_ = X[np.where(kmeans.labels_ == i)[0]]
    r = rank.fit(cluster_)
    clusters_R.append(r)
    print("r: "+str(r))
    U,Sigma,VT = np.linalg.svd(cluster_)
    V = VT.T
    clusters_V.append(V)
    cluster_ = np.matmul(cluster_,V[:,:r])
    weight,center = meepc.fit(cluster_)
    weights.append(weight)
    centers.append(center)
    var1=np.square(cluster_-center)
    var2=np.matmul(weight,var1.T)
    radiis_normal.append(np.sqrt(var2))

9
r: 14
r: 11
r: 9
r: 12
r: 11
r: 11
r: 14
r: 12
r: 11


In [278]:
radiis_normal

[array([0.59626579, 0.47015736, 0.32082496, 0.72596503, 0.46768958,
        0.35940554, 0.61273448, 0.36084025, 0.41857576, 0.37857571,
        0.39010818, 0.3269552 , 0.98936692, 0.44899404, 0.42282276,
        0.41437783, 0.60343679, 0.31570729, 0.39746199, 0.56601394,
        0.45838559, 0.46131958, 0.33861753, 0.37322589, 0.62069013,
        0.52525897, 0.66241104, 0.37366587, 0.32900038, 0.34822849,
        0.604924  , 0.47009113, 0.44587111, 0.4092711 , 0.43552165,
        0.47426905, 0.73477746, 0.87665333, 0.74553001, 0.63621204,
        0.54956499, 0.3933851 , 0.64616864, 0.574723  , 0.91740335,
        0.43823767, 0.46591706, 0.47244253, 0.42758103, 0.45865027,
        0.41792306, 0.29408943, 0.77117869, 0.57398826, 0.43153186,
        0.40541334, 0.40834543, 0.4132845 , 0.39519383, 0.43719859,
        0.30506504, 0.42224487, 0.34556481, 0.38235147, 0.62097769,
        0.5189439 , 0.82136359, 0.48072069, 0.46758452, 0.55030438,
        0.71461908, 0.48720235, 0.47253016, 1.00

In [279]:
X_attack = df_attack.iloc[:,sens].values
X_att = hankel.fit(X_attack,30,0.5)
X_att=X_att.T # size--> X1800
# print(X_att.shape)
# X_att = np.matmul(X_att,V[:,:r])
# var1 = np.square(X_att-center)
# var2 = np.matmul(weight,var1.T)
# radiis_attack = np.sqrt(var2)

radiis_attack = []
print(X_att.shape)
for i in range(optimal_k):
    cluster_ = np.matmul(X_att,clusters_V[i][:,:clusters_R[i]])
    var1=np.square(cluster_-centers[i])
    var2=np.matmul(weights[i],var1.T)
    radiis_attack.append(np.sqrt(var2))

(8, 1800)


In [280]:
radiis_attack

[array([1.83735372, 2.71531984, 1.99483013, 2.16687139, 3.81512481,
        4.02507254, 3.75580229, 2.1411828 ]),
 array([2.64355417, 2.17552569, 3.073501  , 2.38136111, 1.33979069,
        0.50880554, 1.36332588, 2.34494256]),
 array([1.39373061, 1.84918286, 1.22197767, 1.01456929, 2.1219865 ,
        1.94007838, 2.12467947, 1.7275896 ]),
 array([2.14130597, 1.47336547, 1.43733982, 2.84687463, 2.33160357,
        2.09673465, 2.01718079, 1.6459293 ]),
 array([2.08315582, 3.09869745, 0.98585873, 2.85734475, 2.05909755,
        1.82646924, 3.0899175 , 1.58680813]),
 array([0.92720564, 2.68733687, 1.71999384, 0.64752405, 2.45391779,
        1.96412494, 2.32231792, 2.76819512]),
 array([1.24517782, 1.37307593, 1.54767576, 1.58639762, 2.01616727,
        1.90684651, 2.15028549, 1.63268412]),
 array([2.56778837, 2.08712767, 2.84494128, 2.00048891, 0.65313827,
        0.9463114 , 2.53644055, 2.18690496]),
 array([1.38197666, 1.12192168, 1.6207952 , 1.31734448, 1.18180496,
        0.57760891, 

In [281]:
# def calc_radius(X,kmeans,optimal_k):
#     radiis = []
#     weights = []
#     centers = []
#     topR =[ ]
#     cluster_V = []
#     for i in range(optimal_k):
#         cluster_points = X[np.where(kmeans.labels_ == i)[0]]
#         r = rank.fit(X)
#         print('r: ',r)
#         topR.append(r)
#         U,Sigma,VT = np.linalg.svd(cluster_points)
#         V = VT.T
#         cluster_V.append(V[:,:r])
#         cluster_points = np.matmul(cluster_points,V[:,:r])
#         print(cluster_points.shape)
#         weight,center = meepc.fit(cluster_points)
#         weights.append(weight)
#         centers.append(center)
#         var1=np.square(cluster_points-center)
#         var2=np.matmul(weight,var1.T)
#         # max_radius=np.amax(var2)
#         radiis.append(np.sqrt(var2))
#     return radiis,weights,centers,cluster_V,topR

In [282]:
# X_n = df_normal.iloc[:,2].values
# X_n = hankel.fit(X_n,10,0.5)
# X_n = X_n.T
# kmeans,optimal_k = cluster.fit(X_n)
# print(optimal_k)
# radiis_normal = []
# for i in range(optimal_k):
#     cluster_points = X_n[np.where(kmeans.labels_ == i)[0]]
#     r = rank.fit(X_n)
#     print('r: ',r)
#     U,Sigma,VT = np.linalg.svd(cluster_points)
#     V = VT.T
#     cluster_points = np.matmul(cluster_points,V[:,:r])
#     print(cluster_points.shape)
#     weight,center = meepc.fit(cluster_points)
#     var1=np.square(cluster_points-center)
#     var2=np.matmul(weight,var1.T)
#     radiis_normal.append(np.sqrt(var2))
# # radiis_normal,weights_normal,centers_normal,cluster_V_normal,topR_normal = calc_radius(X_n,kmeans,optimal_k)
# # radiis_normal

In [283]:
# X_a = df_attack.iloc[:,2].values
# X_a = hankel.fit(X_a,10,0.5)
# X_a = X_a.T
# kmeans = KMeans(n_clusters=optimal_k, init='k-means++')
# kmeans.fit(X_a)
# radiis_attack,weights_attack,centers_attack,cluster_V_attack,topR_attack = calc_radius(X_a,kmeans,optimal_k)
# radiis_attack

In [284]:
def find_threshold(radii_n,radii_a):
    label = [1]*len(radii_n) + [-1]*len(radii_a)
    label = np.array(label)
    radiis = np.array(list(radii_n)+list(radii_a))
    indices = np.argsort(radiis)
    label = label[indices]
    pos_temp = (1+label)//2
    neg_temp = (1-label)//2
    tp = np.cumsum(pos_temp)
    fp = np.cumsum(neg_temp)
    fn = tp[-1] - tp
    fmeas = 2*tp / (2*tp + fp + fn)
    # print(fmeas)
    idx = np.argmax(fmeas)
    print(indices[idx])
    return radiis[indices[idx]]

In [285]:
# threshold_clusters = find_threshold(radiis_normal,radiis_attack)
threshold_clusters = [0]*optimal_k
for i in range(optimal_k):
    threshold_clusters[i] = find_threshold(radiis_normal[i],radiis_attack[i])
threshold_clusters

73
164
42
79
25
35
7
1
12


[1.0007422522444231,
 1.005451824313275,
 1.011382373997595,
 1.0000049371912534,
 0.9999990958061381,
 1.0253193387357353,
 1.0010335771426015,
 1.001567158907319,
 1.0000019110703573]

In [286]:
df_test=pd.read_csv('~/data/testdata.csv')
y_actual=df_test.iloc[:,-1]
df_test=df_test.iloc[:,sensors]

In [287]:
labels=hankel.fit(np.array(y_actual),30,0.5)
isatt=np.any(labels<0,axis=0)
att_indices=np.where(isatt)[0]
att_indices

array([267, 268, 269, 272, 273])

In [288]:
X_test=df_test.iloc[:,sens].values
Xtest=hankel.fit(X_test,30,0.5)  #size -> 1800x274
Xtest=Xtest.T  #size -> 274x1800
# Xtest = np.matmul(Xtest,V[:,:r])
# var1 = np.square(Xtest-center)
# var2 = np.matmul(weight,var1.T)
# distances = np.sqrt(var2)
# print(Xtest.shape)
distances=[]
for i in range(optimal_k):
    Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) #size -> 274 x R
    var1=np.square(Xtest_cluster-centers[i])
    var2=np.matmul(weights[i],var1.T)
    distance=np.sqrt(var2)
    distances.append(distance)

In [289]:
# check_anomaly = threshold_clusters-distances
# for i in range(len(check_anomaly)):
#     if check_anomaly[i] < 0:
#         print(i)

In [290]:
# distances matrix
stacked_arr=np.vstack(distances)
distances_arr=np.transpose(stacked_arr)
distances_arr #-> 274 X 4

array([[2.20369813, 2.40350954, 1.01885505, ..., 1.5507374 , 1.98248827,
        1.28393453],
       [3.75045397, 1.3521707 , 2.13059185, ..., 1.99135802, 0.59952166,
        1.20402333],
       [4.05756771, 0.52695581, 1.93912898, ..., 1.94140685, 0.95836727,
        0.60493995],
       ...,
       [2.6331654 , 1.81713539, 2.32428037, ..., 1.43435511, 1.74728682,
        0.86548183],
       [1.00440239, 2.41651884, 1.04405778, ..., 0.58157898, 2.06238155,
        0.87161735],
       [1.00639868, 2.49133095, 1.16276084, ..., 0.61702116, 2.16521192,
        0.91807816]])

In [291]:
check_anamoly=threshold_clusters-distances_arr     # positive = inside cluster | negative = outside cluster
check_anamoly

array([[-1.20295588, -1.39805772, -0.00747268, ..., -0.54970383,
        -0.98092111, -0.28393262],
       [-2.74971172, -0.34671888, -1.11920948, ..., -0.99032445,
         0.4020455 , -0.20402142],
       [-3.05682545,  0.47849602, -0.9277466 , ..., -0.94037327,
         0.04319989,  0.39506196],
       ...,
       [-1.63242314, -0.81168357, -1.312898  , ..., -0.43332153,
        -0.74571966,  0.13452009],
       [-0.00366014, -1.41106702, -0.03267541, ...,  0.4194546 ,
        -1.06081439,  0.12838456],
       [-0.00565643, -1.48587912, -0.15137847, ...,  0.38401242,
        -1.16364476,  0.08192375]])

In [292]:
#check for if any datapoint is coming inside the cluster
is_positive=np.any(check_anamoly>0,axis=1)
#indices of those rows
positive_indices=np.where(is_positive)[0]

In [293]:
for i in [[267, 268, 269, 272, 273]]:
    print(distances_arr[i])

[[2.33079984 2.40327708 2.54447239 0.54218207 2.06549508 2.16523537
  1.55544971 2.24958808 1.20499681]
 [0.40106622 3.3116554  1.32694515 1.88817612 1.06669937 1.26241259
  0.97574286 2.88253673 1.23425813]
 [0.92779646 3.11286043 1.58415223 1.9491736  1.83624635 1.18421887
  1.16044997 2.88882037 1.26275161]
 [1.00440239 2.41651884 1.04405778 1.45589546 0.72438997 1.06081396
  0.58157898 2.06238155 0.87161735]
 [1.00639868 2.49133095 1.16276084 1.44987426 1.01779683 1.09345869
  0.61702116 2.16521192 0.91807816]]


In [294]:
for i in range(274):
  if i not in positive_indices:
    print(i)

59
66
89
93
94
150
151
190
246
270
